### Workflow

This time we can use a larger chunk of the original dataset without running into so many memory and runtime problems.

1. Explore and analyze the set of unique characters present in the dataset.
1. Subsample the dataset to take into account the titles only.
1. Implement the tokenization of the dataset and transform the tokens into AllenNLP Instances.
1. Design a RNN with AllenNLP that includes:
   1. an embedding of the tokens,
   1. a seq2seq LSTM layer,
   1. a feed forward layer that outputs probability distribution of the characters
1. Train the model.
1. Evaluate the model by calculating the loss of some sentences and by generating text.

### Bonus

Can you swap a Transformer-based model in for an LSTM? 

### Resources

* [AllenNLP the hard way: Building a Baseline Model](https://jbarrow.ai/allennlp-the-hard-way-2/)
* [Sequential Labeling and Language Modeling (Chapter 5 - Covers allenNLP)]()
* [Atentions and the Transformer (Chapter 8)](https://livebook.manning.com/book/real-world-natural-language-processing/chapter-8/v-5/)
* [Sequence to Sequence Models (Chapter 6)](https://livebook.manning.com/book/real-world-natural-language-processing/chapter-6/v-5/)
* [An In Depth AllenNLP tutorial (Basics to BERT](https://mlexplained.com/2019/01/30/an-in-depth-tutorial-to-allennlp-from-basics-to-elmo-and-bert/)

In [20]:
import pandas as pd
from collections import Counter, defaultdict


### Explore the dataset

In [13]:
TOK_FILE= 'stackexchange_tokenized.csv'
df = pd.read_csv(f'data/{TOK_FILE}')

In [14]:
df.head()

,Unnamed: 0,post_id,parent_id,comment_id,text,category,length,tokenized
0,0,1,NaN,NaN,Eliciting priors from experts,title,29,Eliciting priors from experts
1,1,2,NaN,NaN,What is normality?,title,18,What is normality ?
2,2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title,65,What are some valuable Statistical Analysis op...
3,3,4,NaN,NaN,Assessing the significance of differences in d...,title,58,Assessing the significance of differences in d...
4,4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title,50,The Two Cultures : statistics vs. machine lear...


### Limit Dataset to Titles

In [17]:
df_titles = df[df.category == 'title']

In [18]:
len(df_titles)

91648

In [19]:
len(df)

809166

In [26]:
charCtr = Counter()

def collect_chars(txt):
    global charCtr   
    charCtr.update(list(txt))
        
       

In [28]:
df_titles['text'].apply(collect_chars)
charCtr.most_common(15)

[(' ', 1456196),
 ('e', 981638),
 ('i', 817482),
 ('t', 789104),
 ('a', 737638),
 ('o', 714848),
 ('n', 684648),
 ('r', 600202),
 ('s', 593196),
 ('l', 365504),
 ('d', 298382),
 ('c', 279640),
 ('m', 246208),
 ('u', 224208),
 ('f', 221880)]

### Tokenization

### Testing

In [4]:
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from torch.nn import LSTM

In [5]:
@Model.register('char_lstm')
class CharLSTM(Model):
    pass

In [6]:
EMBEDDING_DIM = 512
HIDDEN_DIM = 256

In [7]:
encoder = PytorchSeq2VecWrapper(
     LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))